## Loading of modules and packages

Activate the Julia environment of the project.

In [ ]:
using Pkg
Pkg.activate("..\\")

Load the necessary packages.

In [ ]:
using ArchGDAL

Load the necessary modules.

In [ ]:
include(".\\Data Gathering\\GroundData.jl")
include(".\\Data Gathering\\SatelliteData.jl")
include(".\\Analysis\\DiluitionAttenuationFactor.jl")
include(".\\Analysis\\Lakes.jl")
include(".\\Analysis\\Plumes.jl")
include(".\\Analysis\\Sediments.jl")


Aliasing of the modules to shorten the code.

In [ ]:
# Modules
const agd = ArchGDAL

# Data gathering
const grdt = GroundData
const stdt = SatelliteData

# Data Analysis
const daf = DiluitionAttenuationFactor 
const lakes = Lakes
const plumes = Plumes
const sediments = Sediments

## Code testing 

#### Data Gathering testing

In [ ]:
grdt.getGroundData( :METEO, grdt.AA, grdt.FVG, grdt.L, grdt.T, grdt.V )

#### Data Analysis testing

##### Initial Setup

In [ ]:
# Digital Terrain Model, the main raster for the analysis, reppresents Veneto
dtm = agd.read("..\\resources\\Analysis test data\\DTM_wgs84.tiff")

Create a source point to test the analysis functions.

In [ ]:
# Coordinates of the source
# lat, lon = (726454.9302346368, 5.025993899219433e6)
lat, lon = (11.930065824163105, 45.425861311724816) # WGS84
# Path to the source shapefile
source_dir = "..\\resources\\Analysis test data\\source_shapefile"
# Directory holding the `source` files
!isdir(source_dir) && mkdir(source_dir)
# Creation of a shapefile containing the point if not already existing
if "source.shp" ∉ readdir(source_dir)
    agd.create(
        source_dir*"\\source.shp",
        driver = agd.getdriver("ESRI Shapefile")
    ) do ds
        agd.createlayer(
            geom = agd.wkbPoint,
            spatialref = agd.importEPSG(4326)
        ) do layer
            agd.createfeature(layer) do feature
                agd.setgeom!(feature, agd.createpoint(lat, lon))
            end
            agd.copy(layer, dataset=ds)
        end
    end
end
source = agd.read(source_dir*"\\source.shp")

Analysis functions execution.

Pollutants leaching.

In [ ]:
daf.run_leach(source, contaminants, concentrations, aquifer_depth,
    acquifer_flow_direction, mean_rainfall, texture, 25, time,
    orthogonal_extension, soil_density, source_thickness, darcy_velocity,
    mixed_zone_depth, decay_coeff, algorithm,
    "..\\resources\\Analysis test data\\Analysis results\\daf.tiff")

Polutants dispersion in lakes.

In [ ]:
lakes.run_lake(source, wind_direction, pollutant_mass, flow_mean_speed, 25,
    hours, fickian_x, fickian_y, λk,
    "..\\resources\\Analysis test data\\Analysis results\\lake.tiff")

Airborne pollutants dispersion from a stack.

In [ ]:
plumes.run_plume(dtm, source, stability, 25, wind_direction, concentration,
    wind_speed, stack_height, gas_speed, stack_diameter, smoke_temperature,
    temperature,
    "..\\resources\\Analysis test data\\Analysis results\\plumes.tiff")

Pollutants dispersion in flowing water.

In [ ]:
sediments.run_sediment(dtm, source, 25, mean_flow_speed, mean_depth,
    x_dispersion_coeff, y_dispersion_coeff, dredged_mass, flow_direction,
    mean_sedimentation_velocity, time, time_intreval,
    current_oscillatory_amplitude, tide,
    "..\\resources\\Analysis test data\\Analysis results\\sediments.tiff")